#**Workshop 12: Statistical Testing**

When working with machine learning models, we often run numerous experiments with various parameters. For instance, performing a grid search can yield a wide array of different results. Even small changes, such as training a model for 10 epochs instead of 9, can lead to different outcomes.

This raises a crucial question: Are these results genuinely different in a meaningful way, or could they merely be due to random chance, such as differences in initialisation or other stochastic processes in the training/validation/testing paradigm?

To address this, we turn to statistical testing. Statistical tests help us determine whether the observed differences in model performance are statistically significant or if they could have arisen by random chance. By applying these tests, we can gain a better understanding of the reliability and robustness of our models' results.

In the following sections, we will use a permutation test and cross validation/confidence intervals to assess the statistical significance of the differences in F1 scores between our models. This approach will allow us to determine whether the observed differences are likely to be real or just random variations.

You'll notice Part 1 is just like workshop 4, where we use the Wisconsin Breast Cancer Dataset.

#**Part 1 [Student, 5mins]: Load and prepare data**

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# To plot even prettier figures
import seaborn as sn

# General data handling (pure numerics are better in numpy)
import pandas as pd

In [ ]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

In [ ]:
xarray = data.data
yarray = data.target
print(xarray.shape)
print(yarray.shape)
fullarray = np.concatenate((xarray,np.reshape(yarray,(-1,1))),axis=1)
print(fullarray.shape)

(569, 30)
(569,)
(569, 31)


In [ ]:
fullarray[:,-1] = 1 - fullarray[:,-1]   # now invert the labels (so that malignant=1)
df = pd.DataFrame(fullarray,columns = list(data.feature_names) + ['target'])

In [ ]:
from sklearn.model_selection import train_test_split

bigtrain_set, test_set = train_test_split(fullarray, test_size=0.2, random_state=42)
train_set, val_set = train_test_split(bigtrain_set, test_size=0.2, random_state=42)

In [ ]:
X_bigtrain = bigtrain_set[:,:-1]
y_bigtrain = bigtrain_set[:,-1]
X_train = train_set[:,:-1]
y_train = train_set[:,-1]
X_test = test_set[:,:-1]
y_test = test_set[:,-1]
X_val = val_set[:,:-1]
y_val = val_set[:,-1]
print([X_train.shape,y_train.shape,X_test.shape,y_test.shape,X_val.shape,y_val.shape])

[(364, 30), (364,), (114, 30), (114,), (91, 30), (91,)]


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

preproc_pl = Pipeline([ ('imputer', SimpleImputer(strategy="median")),
                        ('std_scaler', StandardScaler()) ])

#**Part 2 [Student, 20mins]: Perform Permutation Test**

Run the below cells of code, where you will run three classifiers.

We will run some evaluation metrics and try to assess which is best, and answer the question of how much something has to be different for it to be considered significant (i.e. how much statistical difference between evaluation metrics).

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
rs = np.random.randint(100,size=1)[0]

# Run three classifiers
#-------------------------------------------------------------------------------
#  - The first two are fundamentally the same classifier but just initialised differently.
# Should be equally good on average (but different each time).

#  - The last one should be different on average (last one gives same result each time).

y_val_pred = []
pipes = []

# Initialise with random state
pipes += [ Pipeline([ ('preproc',preproc_pl), ('sgd',SGDClassifier(loss='log_loss',random_state=rs)) ]) ]
pipes[0].fit(X_train,y_train)
y_val_pred +=  [ pipes[0].predict(X_val) ]

pipes += [ Pipeline([ ('preproc',preproc_pl), ('sgd',SGDClassifier(loss='log_loss',random_state=rs+1)) ]) ]
pipes[1].fit(X_train,y_train)
y_val_pred += [ pipes[1].predict(X_val) ]

# This next one is deliberately rather poor (k=200), to show a difference
pipes += [ Pipeline([ ('preproc',preproc_pl), ('knn',KNeighborsClassifier(n_neighbors=200))]) ]
pipes[2].fit(X_train,y_train)
y_val_pred += [ pipes[2].predict(X_val) ]

In [ ]:
# We go to use F1 score to evaluate the performance (just as example, could be whatever metric)
from sklearn.metrics import f1_score

print('Which score is better?')
for n in range(3):
    print(f'Method {n}: {f1_score(y_val,y_val_pred[n]):.3f}')


Which score is better?
Method 0: 0.932
Method 1: 0.958
Method 2: 0.737


In [ ]:
# We can check the specific results of each model
# Because each classifier makes a prediction over lots of different samples, we
# can use this to do some statistics.

y_val_pred[0]

array([1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0.,
       0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 0.])

In [ ]:
# Look at samples where SGD classifiers have not predicted the same.
print(y_val_pred[0]!=y_val_pred[1])

[False False False False False False False False False False False False
 False False False False False False False  True False False False False
 False False False False False False  True False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False]


In [ ]:
# Look at samples where first SGD classifier and KNN classifier have not predicted the same.
print(y_val_pred[0]!=y_val_pred[2])

[False False False False  True False False False False  True  True False
 False False False  True False False False  True False False  True False
 False False False  True False False  True False False False False False
 False  True False False False False False False False False False False
 False False  True False False  True False False False False False False
 False False False  True False False False False False False False False
  True False False False False False  True False  True False False False
 False False False  True False False False]


### mlxtend
We will be using a new library here:
If you are using pip:
* pip install mlxtend
If you are using conda:
* conda install -c conda-forge mlxtend

In [ ]:
from mlxtend.evaluate import permutation_test   # pip install mlxtend

In [ ]:
ntests = 0
# We go to pick up one model
for n1 in range(2):
    # and compare with another
    for n2 in range(n1+1,3):
        # We go to use a  paired permutation test
        p_value = permutation_test(
            y_val_pred[n1], y_val_pred[n2], paired=True,
            # Comparison step
            func=lambda x, y: np.abs(f1_score(y_val,x) - f1_score(y_val,y)),
            method="approximate", seed=0, num_rounds=1000
        )
        ntests += 1

        print(f'P value comparing methods {n1} and {n2}: {p_value:.3f}')
print(f'\nThreshold is {0.05/ntests:.4f}, where P value needs to be *below* this for significance')

P value comparing methods 0 and 1: 0.489
P value comparing methods 0 and 2: 0.002
P value comparing methods 1 and 2: 0.001

Threshold is 0.0167, where P value needs to be *below* this for significance


### Comparing another example


In [ ]:
# What if we now change our KNN model?
y_val_pred = []
pipes = []
p_value = []

# Initialise with random state
pipes += [ Pipeline([ ('preproc',preproc_pl), ('sgd',SGDClassifier(loss='log_loss',random_state=rs)) ]) ]
pipes[0].fit(X_train,y_train)
y_val_pred +=  [ pipes[0].predict(X_val) ]

pipes += [ Pipeline([ ('preproc',preproc_pl), ('sgd',SGDClassifier(loss='log_loss',random_state=rs+1)) ]) ]
pipes[1].fit(X_train,y_train)
y_val_pred += [ pipes[1].predict(X_val) ]

pipes += [ Pipeline([ ('preproc',preproc_pl), ('knn',KNeighborsClassifier(n_neighbors=140))]) ]
pipes[2].fit(X_train,y_train)
y_val_pred += [ pipes[2].predict(X_val) ]

# We go to use F1 score to evaluate the performance (just as example, could be whatever metric)
print('Which score is better?')
for n in range(3):
    print(f'Method {n}: {f1_score(y_val,y_val_pred[n]):.3f}')

Which score is better?
Method 0: 0.932
Method 1: 0.958
Method 2: 0.879


In [ ]:
# Perform permutation test again with new value of k

ntests = 0
# We go to pick up one model
for n1 in range(2):
    # and compare with another
    for n2 in range(n1+1,3):
        # We go to use a  paired permutation test
        p_value = permutation_test(
            y_val_pred[n1], y_val_pred[n2], paired=True,
            # Comparison step
            func=lambda x, y: np.abs(f1_score(y_val,x) - f1_score(y_val,y)),
            method="approximate", seed=0, num_rounds=1000
        )
        ntests += 1

        print(f'P value comparing methods {n1} and {n2}: {p_value:.3f}')
print(f'\nThreshold is {0.05/ntests:.4f}, where P value needs to be *below* this for significance')

P value comparing methods 0 and 1: 0.489
P value comparing methods 0 and 2: 0.173
P value comparing methods 1 and 2: 0.040

Threshold is 0.0167, where P value needs to be *below* this for significance


**Intepretation:** The p-value helps us determine whether the evaluation metric (F1 scores) are likely to come from models with different performance.

* Low p-value: If the p-value is low (typically less than 0.05), it suggests that the F1 scores are significantly different, indicating that the models perform differently.

* High p-value: If the p-value is high, it suggests that the F1 scores are not significantly different, indicating that the models perform similarly.
In essence, a low p-value indicates strong evidence against the null hypothesis (i.e., the models have similar performance), while a high p-value indicates weak evidence against the null hypothesis.



**Some notes and questions**

In this case, we divide the significance level by 3 (hence the 0.05/ntests). Why? When you have 3 models, you need to make three pairwise comparisons: A vs. B, A vs. C, and B vs. C. To illustrate, consider rolling dice: as you roll more dice, the likelihood of getting at least one six increases. Similarly, when performing multiple tests, the likelihood of obtaining a random result that fulfills p < 0.05 also increases. This phenomenon is known as the multiple comparisons problem. To account for this, we adjust our significance threshold to reduce the chances of a Type I error (false positive). Instead of using a significance level of 0.05 for each individual test, we use a corrected threshold of 0.05 divided by the number of tests (3 in this case), resulting in 0.0167. This correction is called the Bonferroni correction. While there are more sophisticated correction methods, Bonferroni is straightforward and intuitively easy to understand.

* Looking at the P-values... are any of them small enough?

* Try modifying the KNN model to have more or less neighbours. How do the p-values change?

* Here is a nice visual explainer: https://www.jwilber.me/permutationtest/

### False Discovery Rate (FDR) Correction

In multiple hypothesis testing, it’s essential to adjust for the increased likelihood of obtaining false positives due to performing multiple comparisons. The example above shows a common method for this adjustment, called the Bonferroni correction. However, an alternative approach that can be used is the False Discovery Rate (FDR) correction, specifically the Benjamini-Hochberg procedure, which can be implemented using the fdrcorrection function from the statsmodels library.

**Purpose:**
The fdrcorrection method is used to control the false discovery rate, which is the expected proportion of false positives among all significant hypotheses. This approach is often preferred over the Bonferroni correction when you want to balance the need to identify true positives while controlling for false positives.

**How It Works:**
* Input P-values: You start with a list of p-values obtained from multiple hypothesis tests.
* FDR Correction: The fdrcorrection function adjusts these p-values to control the false discovery rate. It returns two outputs:
** 'hyptest': A list of Boolean values indicating whether each hypothesis is * significant (True) or not (False) after the correction.
** 'pcorr': A list of the corrected p-values.
* Interpretation: You can compare the corrected p-values (pcorr) to a standard significance threshold (e.g., 0.05) to determine which hypotheses are significant.

**Comparison to Bonferroni Correction:**
* Bonferroni Correction: This method is very conservative, as it adjusts the significance threshold by dividing it by the number of tests. This can reduce the likelihood of false positives but also increases the risk of false negatives (missing true effects).

* FDR Correction: The fdrcorrection method is less conservative compared to Bonferroni. It aims to control the proportion of false positives among the rejected hypotheses, allowing for more true positives to be identified while still controlling for false discoveries.


In [ ]:
from statsmodels.stats.multitest import fdrcorrection

pvals = []
ntests = 0
for n1 in range(2):
    for n2 in range(n1 + 1, 3):
        p_value = permutation_test(
            y_val_pred[n1], y_val_pred[n2], paired=True,
            func=lambda x, y: np.abs(f1_score(y_val, x) - f1_score(y_val, y)),
            method="approximate", seed=0, num_rounds=1000
        )
        ntests += 1
        pvals.append(p_value)

# Apply FDR correction on the list of p-values
hyptest, pcorr = fdrcorrection(pvals, method='p')

print(f"Hypothesis test results (True = different): {hyptest}")
print(f"Corrected P-values (compare to standard 0.05 threshold): {pcorr}")


Hypothesis test results (True = different): [False False False]
Corrected P-values (compare to standard 0.05 threshold): [0.48851149 0.25924076 0.11988012]


#**Part 3 [Student, 10mins]: Perform Cross Validation and Calculate Confidence Intervals**
Another way to assess these statistics is by examining the confidence interval (CI) of the data. Here, we use cross-validation to generate performance metrics (such as the F1 score).

After calculating our scores, we generate confidence intervals. The value '1.96' corresponds to a 95% confidence level. This means that based on the number of times we ran the test (in this case, 5), we can estimate how 'confident' we are that the mean value is accurate. The confidence interval represents the range within which we are 95% certain that the true value lies.

If the confidence intervals of two models overlap, it indicates that we cannot say with certainty that the performance of the two models is distinct.

In [ ]:
# Cross validation results for comparison
#  - the P values above are the best test, but this is a useful comparison

from sklearn.model_selection import cross_validate
from sklearn.metrics import fbeta_score, make_scorer

f1_scorer = make_scorer(fbeta_score, beta=1)

for n in range(3):
    print(f'*** Results for method {n} ***')
    cv_results = cross_validate(pipes[n], X_bigtrain, y_bigtrain, cv=5, return_train_score=True, scoring=f1_scorer)
    # the following is a 95% confidence interval calculation
    # - this is better than standard deviation as overlapping intervals => no stat significance (approx)
    # Note that this is approximate as it assumes a normal distribution
    #  it also does not take into account multiple tests
    #  therefore it implies significance more easily than the P values above (which are better)
    CI = np.std(cv_results['test_score'])*1.96/np.sqrt(cv_results['test_score'].shape[0])
    print(f"  Validation results are {np.mean(cv_results['test_score']):.3f} +/- {CI:.3f}")

*** Results for method 0 ***
  Validation results are 0.933 +/- 0.032
*** Results for method 1 ***
  Validation results are 0.953 +/- 0.019
*** Results for method 2 ***
  Validation results are 0.716 +/- 0.073


#**Part 4 [Student, 10mins]: Adjusting sample size**

When analysing the performance of different machine learning models, the size of the sample (in this demonstration shown by changing the size of the validation set) can significantly impact the statistical significance of the results.

Run the code below. What do you notice?


In [ ]:
from sklearn.model_selection import train_test_split

bigtrain_set, test_set = train_test_split(fullarray, test_size=0.4, random_state=42)

In [ ]:
X_bigtrain = bigtrain_set[:,:-1]
y_bigtrain = bigtrain_set[:,-1]
X_train = train_set[:,:-1]
y_train = train_set[:,-1]
X_test = test_set[:,:-1]
y_test = test_set[:,-1]
X_val = val_set[:,:-1]
y_val = val_set[:,-1]
print([X_train.shape,y_train.shape,X_test.shape,y_test.shape,X_val.shape,y_val.shape])

[(364, 30), (364,), (228, 30), (228,), (91, 30), (91,)]


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

preproc_pl = Pipeline([ ('imputer', SimpleImputer(strategy="median")),
                        ('std_scaler', StandardScaler()) ])

In [ ]:
rs = np.random.randint(100, size=1)[0]

# run three classifiers
#  - the first two should be equally good on average (but different each time)
#  - the last one should be different on average (last one gives same result each time)

y_val_pred = []
pipes = []

pipes += [ Pipeline([ ('preproc',preproc_pl), ('sgd',SGDClassifier(loss='log_loss',random_state=rs)) ]) ]
pipes[0].fit(X_train,y_train)
y_val_pred +=  [ pipes[0].predict(X_val) ]

pipes += [ Pipeline([ ('preproc',preproc_pl), ('sgd',SGDClassifier(loss='log_loss',random_state=rs+1)) ]) ]
pipes[1].fit(X_train,y_train)
y_val_pred += [ pipes[1].predict(X_val) ]

# this next one is deliberately rather poor, to show a difference (n_neighbours is too high)
pipes += [ Pipeline([ ('preproc',preproc_pl), ('knn',KNeighborsClassifier(n_neighbors=200)) ]) ]
pipes[2].fit(X_train,y_train)
y_val_pred += [ pipes[2].predict(X_val) ]

In [ ]:
from sklearn.metrics import f1_score

print('Which score is better?')

for n in range(3):
    print(f'Method {n}: {f1_score(y_val,y_val_pred[n]):.3f}')

Which score is better?
Method 0: 0.958
Method 1: 0.944
Method 2: 0.737


In [ ]:

pvals = []
ntests = 0
for n1 in range(2):
    for n2 in range(n1+1,3):
        p_value = permutation_test(
            y_val_pred[n1], y_val_pred[n2], paired=True,
            func=lambda x, y: np.abs(f1_score(y_val,x) - f1_score(y_val,y)),
            method="approximate", seed=0, num_rounds=1000
        )
        ntests += 1

        pvals += [p_value]

        print(f'P value comparing methods {n1} and {n2}: {p_value:.3f}')
print(f'\nThreshold is {0.05/ntests:.4f}, where P value needs to be *below* this for significance')

P value comparing methods 0 and 1: 1.000
P value comparing methods 0 and 2: 0.001
P value comparing methods 1 and 2: 0.001

Threshold is 0.0167, where P value needs to be *below* this for significance


In [ ]:
# Apply FDR correction on the list of p-values
hyptest, pcorr = fdrcorrection(pvals, method='p')

print(f"Hypothesis test results (True = different): {hyptest}")
print(f"Corrected P-values (compare to standard 0.05 threshold): {pcorr}")


Hypothesis test results (True = different): [False  True  True]
Corrected P-values (compare to standard 0.05 threshold): [1.        0.0014985 0.0014985]
